# Modelo de predicción en OilyGiant

La compañía minera OilyGiant quiere encontrar el mejor lugar para su próximo nuevo pozo. Para encontrar la mejor ubicación se debe realizar lo siguiente: 
- Recolectar los parámetros del pozo de petróleo en la región seleccionada: 
    - Calidad del petróleo y volumen de reservas. 
- Utilizar el modelo de regresión lineal para predecir el volumen de reservas en los nuevos pozos. 
- Seleccionar los pozos de petróleo con los valores estimados más altos. 
- Elegir la región con el mayor beneficio total para los pozos de petróleo seleccionados. 

Se tiene datos sobre las muestras de crudo de tres distintas regiones. En base a la información proporcionada, se debe crear un modelo que ayude a elegir la región con el mayor margen de beneficio, analizando los beneficios y riesgos potencionales utilizando la técnica de *bootstrapping*. 

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Información-general-del-estudio" data-toc-modified-id="Información-general-del-estudio-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Información general del estudio</a></span></li><li><span><a href="#Inicialización" data-toc-modified-id="Inicialización-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Inicialización</a></span><ul class="toc-item"><li><span><a href="#Librerías" data-toc-modified-id="Librerías-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Librerías</a></span></li><li><span><a href="#Cargar-los-datos" data-toc-modified-id="Cargar-los-datos-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Cargar los datos</a></span></li><li><span><a href="#Estudio-de-información-general" data-toc-modified-id="Estudio-de-información-general-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Estudio de información general</a></span></li></ul></li><li><span><a href="#Aplicación-del-modelo-de-predicción" data-toc-modified-id="Aplicación-del-modelo-de-predicción-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Aplicación del modelo de predicción</a></span><ul class="toc-item"><li><span><a href="#Entrenamiento-e-implementación-del-modelo-para-cada-región" data-toc-modified-id="Entrenamiento-e-implementación-del-modelo-para-cada-región-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Entrenamiento e implementación del modelo para cada región</a></span></li></ul></li><li><span><a href="#Volumen-de-reservas-para-el-equilibrio" data-toc-modified-id="Volumen-de-reservas-para-el-equilibrio-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Volumen de reservas para el equilibrio</a></span></li><li><span><a href="#Cálculo-de-ganancias" data-toc-modified-id="Cálculo-de-ganancias-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Cálculo de ganancias</a></span></li><li><span><a href="#Cálculo-de-riesgos-y-ganancias-para-cada-región" data-toc-modified-id="Cálculo-de-riesgos-y-ganancias-para-cada-región-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Cálculo de riesgos y ganancias para cada región</a></span></li><li><span><a href="#Conclusiones" data-toc-modified-id="Conclusiones-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Conclusiones</a></span></li></ul></div>

## Información general del estudio

Los datos de exploración geológica de las tres regiones se almacenan en tres distintos archivos: 

- `geo_data_0.csv` - Región 0
- `geo_data_1.csv` - Región 1
- `geo_data_2.csv` - Región 2

En estos archivos, hay la siguiente información: 
- `id` - identificador único de pozo de petróleo. 
- `f0`,`f1`,`f2` - tres características de los puntos (su significado específico no es importante, pero las características en sí son significativas). 
- `product` - volumen de reservas en el pozo de petróleo (miles de barriles). 

## Inicialización

### Librerías

Se inicia cargando todas las librerías de Python que se utilizarán a lo largo del proyecto.

In [1]:
# cargar todas las librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Cargar los datos

Se carga los datos de los diferentes archivos

In [2]:
# carga del archivo en DataFrame
region_0 = pd.read_csv("/datasets/geo_data_0.csv")
region_1 = pd.read_csv("/datasets/geo_data_1.csv")
region_2 = pd.read_csv("/datasets/geo_data_2.csv")

### Estudio de información general

Comenzamos a revisar el contenido de los archivos. Para esto creamos una función que nos indique el nombre de columnas y el número de filas por la cantidad de columnas. 

In [3]:
# función para enseñar las columnas, número de filas x columnas
def show_columns(file_name, data_frame):
    print(f"Columnas de {file_name}:")
    print(data_frame.columns)
    print("")
    print(f"Filas y columnas de {file_name}: {data_frame.shape}")
    print("\n")

In [4]:
# llamamos a la función para cada archivo
show_columns("region_0", region_0)
show_columns("region_1", region_1)
show_columns("region_2", region_2)

Columnas de region_0:
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

Filas y columnas de region_0: (100000, 5)


Columnas de region_1:
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

Filas y columnas de region_1: (100000, 5)


Columnas de region_2:
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')

Filas y columnas de region_2: (100000, 5)




Los tres archivos tienen la misma cantidad de filas y columnas (100,000 filas y 5 columnas), al igual que los mismos nombres. Ahora veamos de forma rápida las primeras 5 filas, la información general de cada uno de los archivos y un resumen estadístico de los datos de cada tabla. 

In [5]:
# imprimir nombres columnas y número filas x columnas x archivo
oily_giant_list = [region_0, region_1, region_2]

In [6]:
# revisión datos iniciales e impresión de tipos de datos
for index, variable in enumerate(oily_giant_list): 
    print(f"Oily Giant region_{index} info:")
    display(variable.head())
    print("")
    print(variable.info())
    print("")
    print(variable.describe())
    print("")

Oily Giant region_0 info:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.34

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%         1.153055      -4.813172       2.011479      57.085625
75%         8.621015      -1.332816       3.999904     107.813044
max        29.421755      18.73

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.174820       0.130359      59.450441
50%         0.009424      -0.009482       2.484236      94.925613
75%         1.158535       1.163678       4.858794     130.595027
max         7.238262       7.84

Lo importante aquí a resaltar es que la información de cada uno de los archivos de los datos de exploración geológica de cada una de las tres regiones es distinta, y las tres características `f0`, `f1` y `f2` son completamente distintas a nivel estadístico.

Adicional, vemos que todos los archivos están limpios, sin valores ausentes y con datos coherentes entre cada uno de las columnas. Por lo tanto, podemos ya implementar el modelo de predicción para nuestras bases de datos. 

## Aplicación del modelo de predicción

### Entrenamiento e implementación del modelo para cada región

Como tenemos tres bases de datos (las tres regiones), y debemos implementar los mismos pasos para cada uno de las regiones, vamos a crear una función que realice lo mismo. 

Creamos la función `train_and_test` que tiene como argumento el dataframe de cada región. A continuación detallamos los pasos de la función que haría para cada dataframe: 
- Extrae las características y el objetivo del conjunto de datos. 
- Divide el conjunto de datos en conjuntos de entrenamiento y validación (75% entrenamiento y 25% validación).
- Crea un modelo de regresión lineal.
- Entrena el modelo utilizando el conjunto de entrenamiento.
- Realiza predicciones en el conjunto de validación.
- Calcula lo siguiente: 
    - Raíz del error cuadrático medio (RMSE): medida de precisión que indica qué tan cerca están las predicciones del modelo de los valores reales. Cuanto menor sea menor el valor de RMSE mejor será el ajuste del modelo de los datos.  
    - Volumen promedio de las reservas: varía según cada región. 
- Devuelve las predicciones, los valores reales y los cálculos del RMSE y volumen promedio. 

In [7]:
# función de entrenamiento y prueba
def train_and_test(region_data):
    # extraer features y target del conjunto de datos
    features = region_data.drop(['id', 'product'], axis=1)
    target = region_data['product']

    # división en conjuntos de entrenamiento (75%) y validación (25%) .
    X_train, X_valid, y_train, y_valid = train_test_split(features, target, test_size=0.25, random_state=42)

    # modelo de regresión lineal
    model = LinearRegression()

    # entrenamiento del modelo con el conjunto de entrenamiento
    model.fit(X_train, y_train)

    # predicciones conjunto de validación
    predictions = model.predict(X_valid)

    # cálculo (RMSE) para la precisión del modelo
    rmse = mean_squared_error(y_valid, predictions, squared=False)

    # cálculo del volumen promedio de las reservas predichas
    average_volume = predictions.mean()

    # devuelve las predicciones, los valores reales, el RMSE y el volumen promedio
    return predictions, y_valid, rmse, average_volume

Implementamos la función en las tres regiones y verifcamos los valores de RMSE y volumen promedio. 

In [8]:
# obtención de predicciones, valores reales, RMSE y volumen promedio para cada región
predictions_0, y_valid_0, rmse_0, avg_vol_0 = train_and_test(region_0)
predictions_1, y_valid_1, rmse_1, avg_vol_1 = train_and_test(region_1)
predictions_2, y_valid_2, rmse_2, avg_vol_2 = train_and_test(region_2)

In [9]:
# impresión de valores
print("Región 0: RMSE =", rmse_0, "Volumen promedio =", avg_vol_0)
print("Región 1: RMSE =", rmse_1, "Volumen promedio =", avg_vol_1)
print("Región 2: RMSE =", rmse_2, "Volumen promedio =", avg_vol_2)

Región 0: RMSE = 37.756600350261685 Volumen promedio = 92.3987999065777
Región 1: RMSE = 0.890280100102884 Volumen promedio = 68.71287803913762
Región 2: RMSE = 40.14587231134218 Volumen promedio = 94.77102387765939


En base a los siguientes resultados, la región 1 es el que mejor RMSE se obtiene, lo que indica que las predicciones para esta región son más precisas que las otras dos regiones (región 0 y región 2). 

En cambio en la región 0 y 2 son las que mayor volumen promedio de reservas tienen en comparación con la región 1. Esto podría implicar que hay más reservas de petróleo disponibles en estas dos regiones en comparación con la región 1. Si el objetivo es maximizar las reservas de petróleo, entonces la región 0 o la región 2 podrían ser opciones más atractivas.

## Volumen de reservas para el equilibrio

Para realizar el cálculo de todas las ganancias, se debe establecer las variables clave para realizar los cálculos: 
1. `budget`: presupuesto total para el desarrollo de pozos petroleros, es de 100 millones de dólares.
2. `revenue_per_unit`: ingreso generado por un barril de petróleo, es de 4.5 USD. Dado que el volumen de reservas está en miles de barriles, el ingreso por unidad de producto es de 4500 dólares.
3. `wells_to_develop`: número de pozos petroleros que se planea desarrollar, es de 200 pozos.

In [10]:
# almacenamiento de valores en variables
budget = 100000000
revenue_per_unit = 4500
wells_to_develop = 200

En base a estos valores, calculamos el volumen de reservas suficiente para desarrollar un nuevo pozo sin pérdidas, utilizando la siguiente fórmula: 

In [11]:
# cálculo de volumen para un pozo sin pérdidas
break_even_volume = budget / (revenue_per_unit * wells_to_develop)
print("Volumen de reservas necesario para equilibrio:", break_even_volume)

Volumen de reservas necesario para equilibrio: 111.11111111111111


Necesitamos tener un volumen de reservas de 111.11 para poder desarrollar un nuevo pozo sin pérdidas. Ahora comparemos este valor en base al volumen promedio de cada una de las regiones para ver qué región es más factible para el desarrollo de un nuevo pozo. 

In [12]:
# volumen promedio por región
print("Región 0: Volumen promedio =", avg_vol_0)
print("Región 1: Volumen promedio =", avg_vol_1)
print("Región 2: Volumen promedio =", avg_vol_2)

Región 0: Volumen promedio = 92.3987999065777
Región 1: Volumen promedio = 68.71287803913762
Región 2: Volumen promedio = 94.77102387765939


In [13]:
# comparación del volumen necesario para el equilibrio con cada región (0 al 2)
if break_even_volume < avg_vol_0:
    print("Región 0: Factible para el desarrollo de pozos petroleros")
else:
    print("Región 0: No factible para el desarrollo de pozos petroleros")

if break_even_volume < avg_vol_1:
    print("Región 1: Factible para el desarrollo de pozos petroleros")
else:
    print("Región 1: No factible para el desarrollo de pozos petroleros")

if break_even_volume < avg_vol_2:
    print("Región 2: Factible para el desarrollo de pozos petroleros")
else:
    print("Región 2: No factible para el desarrollo de pozos petroleros")

Región 0: No factible para el desarrollo de pozos petroleros
Región 1: No factible para el desarrollo de pozos petroleros
Región 2: No factible para el desarrollo de pozos petroleros


En base a los resultados obtenidos, ninguna de las regiones es factible el desarrollo de pozos petroleros, ya que el volumen promedio de reservas en cada región es menor que el volumen de reservas necesario para alcanzar el punto de equilibrio (111.111 barriles por pozo).

Esto indica que, con los datos actuales, el desarrollo de pozos petroleros en cualquiera de las regiones no sería rentable, ya que el volumen promedio de reservas es insuficiente para cubrir los costos asociados al desarrollo de los pozos.

Sería recomendable realizar un análisis más profundo de los datos y, si es posible, obtener información adicional sobre las reservas en cada región antes de tomar una decisión sobre el desarrollo de pozos petroleros. También se podría ajustar los parámetros del modelo, o incluso probar diferentes enfoques para mejorar las predicciones, y así obtener una mejor estimación de las reservas en cada región.

## Cálculo de ganancias

Necesitamos calcular la ganancia esperada de desarrollar un conjunto de pozos de petróleo en una región específica, basándonos en las predicciones de volumen de reservas y teniendo en cuenta el presupuesto y los ingresos por unidad.

Para esto, construimos la siguiente función `calculate_profit` que contiene los siguientes argumentos: 

1. `predictions`: tipo de dato serie de pandas que contiene las predicciones de volumen de reservas de petróleo para los pozos en una región. Estas predicciones se obtuvieron a partir del modelo de regresión lineal entrenado en los datos de esa región.

2. `target`: tipo de dato serie de pandas que contiene los valores reales (verdaderos) del volumen de reservas de petróleo para los pozos en la misma región. Estos valores se utilizan para comparar con las predicciones y calcular el volumen total de reservas de los pozos seleccionados.

3. `count`: número entero que indica la cantidad de pozos que se deben seleccionar para el desarrollo, basándose en las predicciones más altas. En este caso, se seleccionarán el número de pozos con las mayores predicciones de volumen de reservas, en este caso tiene como número 200.

4. `revenue_per_unit`: número que representa el ingreso generado por cada unidad de volumen de petróleo extraído. Se utiliza para calcular los ingresos totales generados por los pozos seleccionados.

5. `budget`: número que indica el presupuesto total asignado para el desarrollo de los pozos en una región. Se utiliza para calcular la ganancia, restando el presupuesto a los ingresos totales.

6. `sample_size`: número entero, que está predeterminado en 500, que indica cuántos pozos se muestrearán aleatoriamente con reemplazo en cada iteración del proceso de bootstrap. 

7. `bootstrap_size`: número entero, que está predeterminado en 1000, que indica cuántas veces se realizará el proceso de bootstrap. 

In [14]:
def calculate_bootstrap_profit(predictions, target, count, revenue_per_unit, budget, sample_size=500, bootstrap_size=1000):
    # se pasa a list para eliminar el índice anterior
    predictions = pd.Series(list(predictions))
    target = pd.Series(list(target))

    profit_values = []
    # for loop para el bootstrap
    for i in range(bootstrap_size):
        # selecciona 500 pozos al azar en la región
        target_sample = target.sample(sample_size, replace=True)

        # toma las 500 predicciones 
        predictions_sample = predictions.loc[target_sample.index]

        # toma las 200 mejores predicciones
        selected_wells = predictions_sample.sort_values(ascending=False)[:count]

        # obtención de valores reales (target) según los pozos seleccionados
        selected_target = target.reindex(selected_wells.index)

        # cálculo del volumen total de reservas de los pozos seleccionados
        total_volume = selected_target.sum()

        # cálculo de los ingresos multiplicando el volumen total de reservas por el ingreso por unidad
        revenue = total_volume * revenue_per_unit

        # cálculo de la ganancia menos el presupuesto de los ingresos
        profit = revenue - budget
        
        # se añade a lista vacía
        profit_values.append(profit)

    # convierte en un series
    return pd.Series(profit_values)

In [15]:
# se aplica la función para cada región, en base a la información anterior
# tomar en cuenta que 0, 1 y 2 son cada una de las regiones
# almacenamiento de valores en variables las variables: budget = 100000000, revenue_per_unit = 4500, wells_to_develop = 200
bootstrap_profit_0 = calculate_bootstrap_profit(predictions_0, y_valid_0, wells_to_develop, revenue_per_unit, budget)
bootstrap_profit_1 = calculate_bootstrap_profit(predictions_1, y_valid_1, wells_to_develop, revenue_per_unit, budget)
bootstrap_profit_2 = calculate_bootstrap_profit(predictions_2, y_valid_2, wells_to_develop, revenue_per_unit, budget)

# ganancia promedio de cada región
average_profit_0 = bootstrap_profit_0.mean()
average_profit_1 = bootstrap_profit_1.mean()
average_profit_2 = bootstrap_profit_2.mean()

print("Región 0: Ganancia promedio =", average_profit_0)
print("Región 1: Ganancia promedio =", average_profit_1)
print("Región 2: Ganancia promedio =", average_profit_2)

Región 0: Ganancia promedio = 4080507.583958294
Región 1: Ganancia promedio = 4394814.768183932
Región 2: Ganancia promedio = 3737244.797004451


La ganancia promedio para la región 0 y 1 es más de 4 millones. Para el caso de la región 2 la ganancia es de 3.7 millones, ahora en base a estos cálculos, sugerimos qué región es mejor para el desarrollo de pozos petroleros. 

In [16]:
# ganancia máxima entre las tres regiones
max_profit = max(average_profit_0, average_profit_1, average_profit_2)

# qué región tiene la ganancia máxima
if max_profit == average_profit_0:
    chosen_region = "Región 0"
elif max_profit == average_profit_1:
    chosen_region = "Región 1"
else:
    chosen_region = "Región 2"

# resultado
print("Se sugiere la", chosen_region, "para el desarrollo de pozos petroleros, con una ganancia estimada de", round(max_profit, 2))

Se sugiere la Región 1 para el desarrollo de pozos petroleros, con una ganancia estimada de 4394814.77


La Región 1 es la opción más rentable para el desarrollo de pozos petroleros, ya que presenta la mayor ganancia promedio estimada en comparación con las otras dos regiones. En este caso, se estima que la ganancia será de aproximadamente 4,470,199.72 unidades monetarias.

Es importante tener en cuenta que estos resultados se basan en el análisis de datos históricos y en el método de bootstrap para estimar las ganancias en cada región. 

Aunque este enfoque puede proporcionar una buena orientación para la toma de decisiones, hay que resaltar que existe cierta incertidumbre en la predicción de resultados futuros. Para cualquier decicisión que se deba tomar, se debe considerar otros factores que puedan afectar en la toma de decisiones y realizar análisis adicionales.

## Cálculo de riesgos y ganancias para cada región

Perfecto, como ya tenemos el cálculo de las ganancias, veamos ahora el riesgo de perdida. Para esto, se realizará una función que tomará como argumento la serie de valores de ganancia (calculada anteriormente), el cual calcula la ganancia promedio, el intervalo de confianza del 95% y el riesgo de pérdidas para esos valores. 

In [17]:
# cálculo de métricas de ganancia (promedio, intervalo de confianza y riesgo de pérdidas)
def profit_metrics(profit_values):
    # ganancia promedio
    mean_profit = profit_values.mean()  
    # intervalo de confianza del 95%
    confidence_interval = np.percentile(profit_values, [2.5, 97.5])  
    # riesgo de pérdidas en porcentaje
    loss_risk = (profit_values < 0).mean() * 100  
    # devuelve valores calculados
    return mean_profit, confidence_interval, loss_risk

Apliquemos la función para poder calcular las métricas. A partir de esto, se creará una lista llamada `regions` que contiene las métricas de ganancia para cada región. Con esto filtramos las regiones que tienen un riesgo de pérdidas inferior al 2.5% y almacenamos el resultado en una lista llamada `filtered_regions`. Seleccionamos la región con la ganancia promedio más alta entre las regiones filtradas, y almacenamos el resultado en una variable llamada `best_region`. Encontramos el índice de la región seleccionada en la lista original regions y lo almacenamos en una variable llamada `region_index`.

En base a esto, podemos mostrar la región seleccionada, incluyendo la ganancia promedio, el intervalo de confianza del 95% y el riesgo de pérdidas.

In [18]:
# métricas de ganancia para cada región
profit_metrics_0 = profit_metrics(bootstrap_profit_0)
profit_metrics_1 = profit_metrics(bootstrap_profit_1)
profit_metrics_2 = profit_metrics(bootstrap_profit_2)

# lista de métricas de ganancia para cada región
regions = [profit_metrics_0, profit_metrics_1, profit_metrics_2]

# filtro de regiones que tienen un riesgo de pérdidas inferior al 2.5%
filtered_regions = [region for region in regions if region[2] < 2.5]

# región con ganancia promedio más alta 
best_region = max(filtered_regions, key=lambda x: x[0])

# encontrar el índice de la región seleccionada 
region_index = regions.index(best_region)

# mostrar información de la región seleccionada
print(f"Se sugiere la Región {region_index} para el desarrollo de pozos petroleros.")
print(f"Ganancia promedio: {best_region[0]:.2f}")
print(f"Intervalo de confianza del 95%: {best_region[1]}")
print(f"Riesgo de pérdidas: {best_region[2]:.2f}%")

Se sugiere la Región 1 para el desarrollo de pozos petroleros.
Ganancia promedio: 4394814.77
Intervalo de confianza del 95%: [ 559527.65029479 8217835.45697761]
Riesgo de pérdidas: 0.80%


La Región 1 cumple con el criterio de riesgo de pérdida inferior al 2.5%, lo que indica que esta región tiene un riesgo de pérdida aceptablemente bajo según el criterio establecido. Por lo tanto, se concluye lo siguiente:

- **Región seleccionada**: La Región 1 es la sugerida para el desarrollo de pozos petroleros, ya que tiene un riesgo de pérdidas del 1.3%, que es inferior al límite del 2.5%. Además, presenta una ganancia promedio de 4,470,199.72 y un intervalo de confianza del 95% entre 451,393.24 y 8,494,111.57.

- **Evaluación de riesgo-beneficio**: es importante revisar el riesgo y el beneficio al tomar decisiones de inversión en la industria del petróleo y el gas. La Región 1 parece ser una opción sólida en función de estos criterios.

## Conclusiones

En este proyecto se llevó a cabo varias etapas para evaluar el potencial de desarrollo de pozos petroleros en tres regiones diferentes, con el objetivo de seleccionar la mejor región para invertir. 

Se analizó y procesó los datos de producción de petróleo en las tres regiones para prepararlos para su uso en modelos de aprendizaje automático.

Se entrenó el modelo de regresión lineal para cada región con el fin de predecir la producción de petróleo en los pozos. Luego, se utilizó el modelo para hacer predicciones en los conjuntos de datos de validación.

Se implementó una función de beneficio para calcular las ganancias esperadas de desarrollar pozos petroleros en cada región, teniendo en cuenta los costos y las predicciones de producción.

Se aplicó el método de bootstrapping para estimar la distribución de ganancias en cada región y se calculó el intervalo de confianza del 95% y el riesgo de pérdida para cada una.

Se estableció un criterio de riesgo de pérdida inferior al 2.5% para seleccionar la mejor región para invertir. La región que cumplió con el criterio fue la Región 1 porque tenía un riesgo de pérdida del 1.30%. 

Finalmente la información de este proyecto proporcionó información valiosa sobre el rendimiento y el riesgo asociados con las inversiones en cada región. Esto puede servir para recomendar a la empresa e indicar qué región invertir para los pozos petroleros, tomando en cuenta que siempre hay otros factores que pueden influir. 